In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from selenium import webdriver
from datetime import date    
today = date.today().isoformat()

import warnings
warnings.simplefilter('ignore')

O-Rtg (need to sum pts,fga,orb,to,fta):

Possessions: field goals attempted - offensive rebounds + turnovers + (0.4 x free throws attempted) = total number of possessions for the season to calculate total team possessions.

O-Rtg = Points / Possessions

D-Rtg (same vars but opp.): D-Rtg = Opp Points / Opp.Poss

In [ ]:
train = pd.read_csv('2014-2018_NBA_Games_RA.csv')

In [ ]:
def rollingMetrics(df):
    
    #create avgs
    for column in df.iloc[:,12:]:
        newC = train[column].rolling(10).mean().shift(1)
        colname = 'mean_' + train[column].name
        train[colname] = newC
    
    #create sums
    for column in train.iloc[:,12:]:
        newC = train[column].rolling(10).sum().shift(1)
        colname = 'sum_' + train[column].name
        train[colname] = newC
    
    #Drop win sum and sum_mean columns, and game specific columns used for aggregations
    train.drop(['sum_Win','sum_mean_Diff','sum_mean_Win','sum_mean_PTS','sum_mean_FGM','sum_mean_FGA','sum_mean_3PM',
                'sum_mean_3PA','sum_mean_FTM','sum_mean_FTA','sum_mean_ORB','sum_mean_TRB','sum_mean_Assists','sum_mean_Steals',
                'sum_mean_Blocks','sum_mean_Turnovers','sum_mean_TotalFouls','FGM','FGA','3PM','3PA','FTM','FTA','ORB','TRB',
                'Assists','Blocks','Steals','Turnovers','TotalFouls',
                'sum_mean_Opp.ORB','sum_mean_Opp.Turnovers','sum_mean_Opp.FTA','sum_mean_Opp.FGA','sum_mean_Opp.PTS',
                'mean_Opp.ORB','mean_Opp.Turnovers','mean_Opp.FTA','mean_Opp.FGA',
                'mean_Opp.PTS','Opp.ORB','Opp.Turnovers','Opp.FTA','Opp.FGA','Opp.PTS'],axis=1,inplace=True)
    
    #drop rows used for first x games seasons -- necessary for first predictions (these are set to RA_ROW = NA)
    train.dropna(inplace=True)
    
    #drop RA_ROW
    train.drop(['RA_ROW'],inplace=True,axis=1)
    
    return train

In [ ]:
avg_sum = rollingMetrics(train)

In [ ]:
#join opponent data after aggregation of metrics
oppFrame = avg_sum.copy()
oppFrame.drop(['Date','Team','Opponent','Month','Year','Game','Home','B2B','Diff','Win','PTS','Opp.B2B'],axis=1,inplace=True)

In [ ]:
#bring in original data frame to join on tm ID and Opp Id
teamFrame = avg_sum.copy()
combined = teamFrame.merge(oppFrame,left_on='Tm.Id',right_on='Opp.Id',suffixes=("","_Opp"))

#Drop columns used for matching
combined.drop(['Tm.Id_Opp','Opp.Id_Opp'],axis=1,inplace=True)

del oppFrame,teamFrame,avg_sum

In [ ]:
# create variables
def createvars(df):
    
        #Create % columns
        
    df['FG%'] = df['sum_FGM']/df['sum_FGA']
    df['3P%'] = df['sum_3PM']/df['sum_3PA']
    df['FT%'] = df['sum_FTM']/df['sum_FTA']

    df['FG%_Opp'] = df['sum_FGM_Opp']/df['sum_FGA_Opp']
    df['3P%_Opp'] = df['sum_3PM_Opp']/df['sum_3PA_Opp']
    df['FT%_Opp'] = df['sum_FTM_Opp']/df['sum_FTA_Opp']

       #Create Assist % and Ast/TO ratio
    
    df['AstTO'] = df['mean_Assists']/df['mean_Turnovers']
    df['Ast%'] = df['mean_Assists']/df['mean_FGM']
    
    df['AstTO_Opp'] = df['mean_Assists_Opp']/df['mean_Turnovers_Opp']
    df['Ast%_Opp'] = df['mean_Assists_Opp']/df['mean_FGM_Opp']    
    
    
        #Create offensive rating for team / opponent
        
    tm_pos = df['sum_FGA'] - df['sum_ORB'] + df['sum_Turnovers'] + (0.4*df['sum_FTA']) 
    df['OFF_Rtng'] = (df['sum_PTS']/tm_pos)*100
    
    opp_pos = df['sum_FGA_Opp'] - df['sum_ORB_Opp'] + df['sum_Turnovers_Opp'] + (0.4*df['sum_FTA_Opp'])
    df['OFF_Rtng_Opp'] = (df['sum_PTS_Opp']/opp_pos)*100
    
        #Create defensive rating for team / opponent
        
    tm_pos = df['sum_Opp.FGA'] - df['sum_Opp.ORB'] + df['sum_Opp.Turnovers'] + (0.4*df['sum_Opp.FTA']) 
    df['DEF_Rtng'] = (df['sum_Opp.PTS']/tm_pos)*100
    
    opp_pos = df['sum_Opp.FGA_Opp'] - df['sum_Opp.ORB_Opp'] + df['sum_Opp.Turnovers_Opp'] + (0.4*df['sum_Opp.FTA_Opp'])
    df['DEF_Rtng_Opp'] = (df['sum_Opp.PTS_Opp']/opp_pos)*100    
    
        #Create net rating for team/opponent
    
    df['NET_Rtng'] = df['OFF_Rtng'] - df['DEF_Rtng']
    df['NET_Rtng_Opp'] = df['OFF_Rtng_Opp'] - df['DEF_Rtng_Opp']
    
        #drop extra columns - sum columns
    cols = [c for c in df.columns if c[:3] == 'sum']
    df.drop(cols,axis=1,inplace=True)
    
        #drop extra columns - some traditional metrics not needed anymore
    
    return df

In [ ]:
combined = createvars(combined)
#combined.to_csv('final.csv',index=False)